In [ ]:
!pip install -qU langgraph langchain langchain-community langchainhub langchain-core langchain_ollama ddgs
from langchain_ollama.chat_models import ChatOllama
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
import uuid
import os
import sys
import re
sys.path.insert(0, '/home/bfhgfe/agent')
import Tools.search
from Tools.search import search
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    AnyMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately

########### < AGENT_DEFINITION > ###########

llm = ChatOllama(model="qwen3:1.7b")
memory = MemorySaver()

whoaminow = """
You are a helpful ai assistant.
"""

taskrules = """

"""
                      
etiquette = str.format(f"""
User: {whoaminow}
System: {taskrules}
""", whoaminow, taskrules)

agent = create_react_agent(
    model=llm,
    tools=[search],
    prompt=etiquette,
    checkpointer = memory
)

messages = []
uuidVal = uuid.uuid4()
config = {
            "recursion_limit": 200,
            "configurable": {"thread_id": str(uuidVal)},
            "repetition penalty": 1.08,
            "DRY": 1
        }
########### </ AGENT_DEFINITION> ###########

########### < Workflow > ###########



########### </ Workflow > ###########


########### < CHAT_PART > ###########
username = input("User name: ")

while True:
    try:
        user_input = input("User: ")
        messages = HumanMessage("<sender>"+username+"</sender><message>"+user_input+"</message>")
        result = await agent.ainvoke({"messages": messages}, think=False)
        for msg in result["messages"]:
            msg.pretty_print()
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
    except EOFError:
        # fallback if input() is not available
        print('eof\n')
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        break
########### </ CHAT_PART > ###########
        